# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [375]:
# Your solution goes here
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [373]:
def scrape_imdb(title_type, user_rating, start_year, end_year):
    url = f'https://www.imdb.com/list/ls025598828/?sort=list_order,asc&st_dt=&mode=detail&page=1' \
          f'&title_type={title_type}&user_rating={user_rating}%2C&release_date={start_year}%2C{end_year}&ref_=ttls_ref_yr'
      
    dictionary_imdb = {
        "number": [], 
        "title": [], 
        "year": [], 
        "rating": [], 
        "runtime": [],
        "genre": [],
        "description": [],
        "director": [],
        "stars": [],
        "votes": [],
        "gross": []
    }

    df_results = pd.DataFrame()
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find_all(class_='lister-item-content')

        for x in range(len(results)):
            # Movie Number
            movie_nr = results[x].find('span', class_='lister-item-index unbold text-primary').getText().rstrip('.')
            dictionary_imdb["number"].append(movie_nr)

            # Movie Title
            title_element = results[x].find('h3', class_='lister-item-header').a
            movie_title = title_element.getText()
            dictionary_imdb["title"].append(movie_title)

            # Year of Release
            year_release = results[x].find('span', class_='lister-item-year text-muted unbold').getText().strip('()')
            dictionary_imdb["year"].append(year_release)

            # IMDb Rating
            rating_of_movie = results[x].find('span', class_='ipl-rating-star__rating').getText()
            dictionary_imdb["rating"].append(rating_of_movie)

            # Runtime
            runtime_of_movie = results[x].find('span', class_='runtime').getText()
            dictionary_imdb["runtime"].append(runtime_of_movie)

            # Genre
            genre_of_movie = results[x].find('span', class_='genre').getText().strip()
            dictionary_imdb["genre"].append(genre_of_movie)

            # Description
            description_of_movie = results[x].find_all('p')[1].getText().strip()
            dictionary_imdb["description"].append(description_of_movie)

            # Director
            director_of_movie = results[x].find_all('p')[2].find('a').getText()
            dictionary_imdb["director"].append(director_of_movie)

            # Stars
            stars_of_movie = [star.getText() for star in results[x].find_all('p')[2].find_all('a')[1:]]
            stars_string = ', '.join(stars_of_movie)
            dictionary_imdb["stars"].append(stars_string)

            # Votes
            votes_of_movie = results[x].find_all('p')[3].find('span', attrs={'name': 'nv'}).getText()
            dictionary_imdb["votes"].append(votes_of_movie)

            # Gross
            gross_of_movie = results[x].find_all('p')[3].find('span', attrs={'name': 'nv'}).getText()
            dictionary_imdb["gross"].append(gross_of_movie)

        df_results = pd.DataFrame(dictionary_imdb)
    else:
        print('Negative Response from server')

    return df_results

In [374]:
title_type = 'movie'
user_rating= '7.5'
start_year = '1990'
end_year = '1992'
df_results = scrape_imdb(title_type,user_rating,start_year,end_year)
df_results.head(20)

,number,title,year,rating,runtime,genre,description,director,stars,votes,gross
0,1,Reservoir Dogs,1992,8.3,99 min,"Crime, Thriller",When a simple jewelry heist goes horribly wron...,Quentin Tarantino,"Harvey Keitel, Tim Roth, Michael Madsen, Chris...","1,087,474","1,087,474"
1,2,El silencio de los corderos,1991,8.6,118 min,"Crime, Drama, Thriller",A young F.B.I. cadet must receive the help of ...,Jonathan Demme,"Jodie Foster, Anthony Hopkins, Scott Glenn, Te...","1,547,075","1,547,075"
2,3,Terminator 2: El juicio final,1991,8.6,137 min,"Action, Adventure, Sci-Fi","A cyborg, identical to the one who failed to k...",James Cameron,"Arnold Schwarzenegger, Linda Hamilton, Edward ...","1,173,150","1,173,150"
3,4,Uno de los nuestros,1990,8.7,145 min,"Biography, Crime, Drama",The story of Henry Hill and his life in the ma...,Martin Scorsese,"Robert De Niro, Ray Liotta, Joe Pesci, Lorrain...","1,256,268","1,256,268"
4,5,El padrino parte III,1990,7.6,162 min,"Crime, Drama","Follows Michael Corleone, now in his 60s, as h...",Francis Ford Coppola,"Al Pacino, Diane Keaton, Andy Garcia, Talia Shire","423,790","423,790"
5,6,Algunos hombres buenos,1992,7.7,138 min,"Drama, Thriller",Military lawyer Lieutenant Daniel Kaffee defen...,Rob Reiner,"Tom Cruise, Jack Nicholson, Demi Moore, Kevin ...","287,114","287,114"
6,7,JFK: Caso abierto,1991,8,189 min,"Drama, History, Thriller",New Orleans District Attorney Jim Garrison dis...,Oliver Stone,"Kevin Costner, Gary Oldman, Jack Lemmon, Walte...","169,829","169,829"
7,8,Los chicos del barrio,1991,7.8,112 min,"Crime, Drama",Follows the lives of three young males living ...,John Singleton,"Cuba Gooding Jr., Laurence Fishburne, Hudhail ...","154,484","154,484"
8,9,Sin perdón,1992,8.2,130 min,"Drama, Western",Retired Old West gunslinger William Munny relu...,Clint Eastwood,"Clint Eastwood, Gene Hackman, Morgan Freeman, ...","435,892","435,892"
9,10,Desafío total,1990,7.5,113 min,"Action, Adventure, Sci-Fi",When a man goes in to have virtual vacation me...,Paul Verhoeven,"Arnold Schwarzenegger, Sharon Stone, Michael I...","354,244","354,244"


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here